In [5]:
import json
import psycopg2
from psycopg2.extras import RealDictCursor
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class ActorDatabase:
    def __init__(self, host="localhost", database="actors_db", user="your_username", password="your_password"):
        """Initialize database connection parameters"""
        self.connection_params = {
            'host': "localhost",
            'database': "Celebrities_data",
            'user': "postgres",
            'password': "Ishan@allen04"
        }
        self.conn = None
        self.cursor = None
    
    def connect(self):
        """Establish database connection"""
        try:
            self.conn = psycopg2.connect(**self.connection_params)
            self.cursor = self.conn.cursor(cursor_factory=RealDictCursor)
            logger.info("Database connection established")
        except Exception as e:
            logger.error(f"Error connecting to database: {e}")
            raise
    
    def create_tables(self):
        """Create the actors table with proper schema"""
        create_table_query = """
        CREATE TABLE IF NOT EXISTS actors (
            id SERIAL PRIMARY KEY,
            rank SERIAL,
            name VARCHAR(255) NOT NULL,
            imdb_id VARCHAR(50) UNIQUE,
            image_url_original TEXT,
            image_url_hd TEXT,
            roles TEXT[],
            known_for VARCHAR(500),
            known_for_url TEXT,
            description TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        
        -- Create indexes for better performance
        CREATE INDEX IF NOT EXISTS idx_actors_rank ON actors(rank);
        CREATE INDEX IF NOT EXISTS idx_actors_name ON actors(name);
        CREATE INDEX IF NOT EXISTS idx_actors_imdb_id ON actors(imdb_id);
        """
        
        try:
            self.cursor.execute(create_table_query)
            self.conn.commit()
            logger.info("Tables created successfully")
        except Exception as e:
            logger.error(f"Error creating tables: {e}")
            self.conn.rollback()
            raise
    
    def insert_actor(self, actor_data):
        """Insert a single actor record"""
        insert_query = """
        INSERT INTO actors (rank, name, imdb_id, image_url_original, image_url_hd, 
                           roles, known_for, known_for_url, description)
        VALUES (%(rank)s, %(name)s, %(imdb_id)s, %(image_url_original)s, 
                %(image_url_hd)s, %(roles)s, %(known_for)s, %(known_for_url)s, %(description)s)
        ON CONFLICT (imdb_id) DO UPDATE SET
            rank = EXCLUDED.rank,
            name = EXCLUDED.name,
            image_url_original = EXCLUDED.image_url_original,
            image_url_hd = EXCLUDED.image_url_hd,
            roles = EXCLUDED.roles,
            known_for = EXCLUDED.known_for,
            known_for_url = EXCLUDED.known_for_url,
            description = EXCLUDED.description;
        """
        
        try:
            self.cursor.execute(insert_query, actor_data)
            return True
        except Exception as e:
            logger.error(f"Error inserting actor {actor_data.get('name', 'Unknown')}: {e}")
            return False
    
    def load_json_file(self, file_path):
        """Load and parse JSON file"""
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
            logger.info(f"Successfully loaded {len(data)} records from {file_path}")
            return data
        except Exception as e:
            logger.error(f"Error loading JSON file: {e}")
            raise
    
    def insert_actors_from_json(self, json_file_path):
        """Main method to insert all actors from JSON file"""
        try:
            # Load JSON data
            actors_data = self.load_json_file(json_file_path)
            
            successful_inserts = 0
            failed_inserts = 0
            
            # Insert each actor
            for actor in actors_data:
                if self.insert_actor(actor):
                    successful_inserts += 1
                else:
                    failed_inserts += 1
            
            # Commit all changes
            self.conn.commit()
            
            logger.info(f"Insertion complete: {successful_inserts} successful, {failed_inserts} failed")
            return successful_inserts, failed_inserts
            
        except Exception as e:
            logger.error(f"Error during bulk insertion: {e}")
            self.conn.rollback()
            raise
    
    def get_actor_count(self):
        """Get total number of actors in database"""
        try:
            self.cursor.execute("SELECT COUNT(*) FROM actors;")
            count = self.cursor.fetchone()['count']
            return count
        except Exception as e:
            logger.error(f"Error getting actor count: {e}")
            return None
    
    def close_connection(self):
        """Close database connection"""
        if self.cursor:
            self.cursor.close()
        if self.conn:
            self.conn.close()
        logger.info("Database connection closed")

def main():
    """Main execution function"""
    # Database configuration
    db_config = {
        'host': "localhost",
        'database': "Celebrities_data",
        'user': "postgres",
        'password': "Ishan@allen04"
    }
    
    # JSON file path
    json_file_path = 'refined_actors_4.json'
    
    # Initialize database handler
    db = ActorDatabase(**db_config)
    
    try:
        # Connect to database
        db.connect()
        
        # Create tables
        db.create_tables()
        
        # Insert actors from JSON
        successful, failed = db.insert_actors_from_json(json_file_path)
        
        # Display results
        total_actors = db.get_actor_count()
        print(f"\n=== INSERTION SUMMARY ===")
        print(f"Successful insertions: {successful}")
        print(f"Failed insertions: {failed}")
        print(f"Total actors in database: {total_actors}")
        
    except Exception as e:
        logger.error(f"Application error: {e}")
    finally:
        db.close_connection()

if __name__ == "__main__":
    main()


INFO:__main__:Database connection established
INFO:__main__:Tables created successfully
INFO:__main__:Successfully loaded 205 records from refined_actors_4.json
INFO:__main__:Insertion complete: 205 successful, 0 failed
INFO:__main__:Database connection closed



=== INSERTION SUMMARY ===
Successful insertions: 205
Failed insertions: 0
Total actors in database: 940


In [6]:
import psycopg2
from psycopg2.extras import RealDictCursor

def connect_to_database():
    """Establish connection to PostgreSQL database"""
    try:
        connection = psycopg2.connect(
            host="localhost",
            database="Celebrities_data",  # Replace with your database name
            user="postgres",  # Replace with your username
            password="Ishan@allen04",  # Replace with your password
            port=5432
        )
        
        print("Successfully connected to the database!")
        return connection
        
    except Exception as e:
        print(f"Error connecting to database: {e}")
        return None

# Usage
conn = connect_to_database()
if conn:
    cursor = conn.cursor(cursor_factory=RealDictCursor)
    # Your database operations here
    conn.close()


Successfully connected to the database!


In [7]:
def get_celebrity_data(actor_id):
    """
    Fetch celebrity data from database using actor ID
    
    Args:
        actor_id (str): IMDB actor ID (e.g., 'nm0000093')
    
    Returns:
        dict: Celebrity data if found, None otherwise
    """
    conn = connect_to_database()
    if not conn:
        print("Failed to connect to database")
        return None
    
    cursor = conn.cursor(cursor_factory=RealDictCursor)
    
    try:
        # SQL query to fetch celebrity data
        query = "SELECT * FROM actors WHERE imdb_id = %s;"
        cursor.execute(query, (actor_id,))
        result = cursor.fetchone()
        
        if result:
            celebrity_data = dict(result)
            print(f"Found celebrity: {celebrity_data.get('name')}")
            return celebrity_data
        else:
            print(f"No celebrity found with ID: {actor_id}")
            return None
            
    except Exception as e:
        print(f"Error fetching celebrity data: {e}")
        return None
    
    finally:
        cursor.close()
        conn.close()




In [8]:
import numpy as np
embeddings_db = np.load(r"C:\Users\ishan\OneDrive\Desktop\face recog\embeddings1_json.npy")  # shape (N, 512)
labels_db = np.load(r"C:\Users\ishan\OneDrive\Desktop\face recog\labels1_json.npy")          # shape (N,)


In [21]:
import os
import cv2
import torch
import pickle
import numpy as np
import face_recognition
from ultralytics import YOLO
from keras_facenet import FaceNet

embedder=FaceNet()

# === Paths ===
encodings_path = r"C:\Users\ishan\OneDrive\Desktop\face recog\workfromstart\new_encodings.pickle"
embeddings_path = r"C:\Users\ishan\OneDrive\Desktop\face recog\workfromstart\embeddingss1_json.npy"
labels_path = r"C:\Users\ishan\OneDrive\Desktop\face recog\workfromstart\labelss1_json.npy"
image_path = r"C:\Users\ishan\Downloads\OIP (3).jpeg"
yolo_model_path = r"C:\Users\ishan\Downloads\yolov8n-face.pt"
model_weights_path = r"C:\Users\ishan\OneDrive\Desktop\face recog\face_match_model.pth"

# === Load encodings and data ===
with open(encodings_path, "rb") as f:
    data = pickle.load(f)

face_encodings_128 = data["encodings"]     # shape: (N, 128)
face_labels = np.array(data["names"])      # shape: (N,)

embeddings_512 = np.load(embeddings_path)  # shape: (N, 512)
labels_512 = np.load(labels_path)          # shape: (N,)

# === Define MLP model ===
class MLP(torch.nn.Module):
    def __init__(self, input_dim=1024):
        super(MLP, self).__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(input_dim, 256),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(128, 1),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP(input_dim=1024)
model.load_state_dict(torch.load(model_weights_path, map_location=device))
model.to(device)
model.eval()

# === Load YOLO model ===
yolo_model = YOLO(yolo_model_path)

# === Load and prepare image ===
image = cv2.imread(image_path)
if image is None:
    raise ValueError("Failed to load image")

image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
results = yolo_model(image_rgb)[0]


recognized_count = 0

for box in results.boxes:
    conf = float(box.conf[0])
    if conf < 0.5:
        continue

    x1, y1, x2, y2 = map(int, box.xyxy[0])
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(image.shape[1], x2), min(image.shape[0], y2)

    face_location = (y1, x2, y2, x1)
    encodings = face_recognition.face_encodings(image_rgb, known_face_locations=[face_location])

    name = "Unknown"
    if encodings:
        face_encoding = encodings[0]  # shape: (128,)
        
        distances = face_recognition.face_distance(face_encodings_128, face_encoding)
        top5_indices = distances.argsort()[:5]

        # === Prepare face crop for 512-d embedding ===
        face_crop = image_rgb[y1:y2, x1:x2]
        face_crop = cv2.resize(face_crop, (160, 160))
        query_embedding = embedder.embeddings([face_crop])[0]  # shape: (512,)

        scores = []

        for idx in top5_indices:
            label = face_labels[idx]
            embedding_candidate = embeddings_512[idx]

            combined = np.concatenate([query_embedding, embedding_candidate])  # shape: (1024,)
            input_tensor = torch.tensor(combined, dtype=torch.float32).unsqueeze(0).to(device)

            with torch.no_grad():
                score = model(input_tensor).item()

            scores.append((score, label))
            break

        # === Pick the best label based on MLP score ===
        scores.sort(reverse=True, key=lambda x: x[0])
        best_score, best_label = scores[0]

        
        name = best_label
        actor_id = best_label.split('_')[0]

        conn = connect_to_database()
        if conn:
                cursor = conn.cursor(cursor_factory=RealDictCursor)
                celebrity_info = get_celebrity_data(actor_id)

                print(f"\n[INFO] Match Found:")
                print(f"Actor Name: {celebrity_info['name']}")
                print(f"IMDB_ID: {celebrity_info['imdb_id']}")
                print(f"Known for: {celebrity_info['known_for']}")
                print(f"About Actor: {celebrity_info['description']}\n")

                conn.close()

        recognized_count += 1
        

    # === Draw result ===
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 1)
    cv2.putText(image, name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

print(f"Recognized {recognized_count} face(s)")

# === Show the result ===
cv2.imshow("YOLO + Face Recognition + MLP", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


INFO:keras_facenet.embedding_model:Loading weights.
INFO:keras_facenet.utils:Looking for C:\Users\ishan/.keras-facenet\20180402-114759\20180402-114759-weights.h5



0: 416x640 1 face, 79.4ms
Speed: 7.7ms preprocess, 79.4ms inference, 3.3ms postprocess per image at shape (1, 3, 416, 640)


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Successfully connected to the database!
Successfully connected to the database!
Found celebrity: Johnny Depp

[INFO] Match Found:
Actor Name: Johnny Depp
IMDB_ID: nm0000136
Known for: Pirates of the Caribbean: The Curse of the Black Pearl (2003)
About Actor: John Christopher "Johnny" Depp II was born on June 9, 1963 in Owensboro, Kentucky, to Betty Sue Palmer (nÃ©e Wells), a waitress, and John Christopher Depp, a civil engineer. He was raised in Florida. He dropped out of school when he was 15, and fronted a series of music-garage bands, including one named 'The Kids'. When he married Lori A. Depp, he took a job as a ballpoint-pen salesman to support himself and his wife. A visit to Los Angeles, California, with his wife, however, happened to be a blessing in disguise, when he met up with actor , who advised him to turn to acting, which culminated in Depp's film debut in the low-budget horror film, , where he played a teenager who falls prey to dream

In [25]:
import os
import cv2
import torch
import pickle
import numpy as np
import face_recognition
from ultralytics import YOLO
from keras_facenet import FaceNet
from psycopg2.extras import RealDictCursor

# === Load FaceNet embedder and MLP model once ===
embedder = FaceNet()

class MLP(torch.nn.Module):
    def __init__(self, input_dim=1024):
        super(MLP, self).__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(input_dim, 256),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(128, 1),
            torch.nn.Sigmoid()
        )
    def forward(self, x):
        return self.net(x)

# === Global config ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encodings_path = r"C:\Users\ishan\OneDrive\Desktop\face recog\workfromstart\new_encodings.pickle"
embeddings_path = r"C:\Users\ishan\OneDrive\Desktop\face recog\workfromstart\embeddingss1_json.npy"
labels_path = r"C:\Users\ishan\OneDrive\Desktop\face recog\workfromstart\labelss1_json.npy"
yolo_model_path = r"C:\Users\ishan\Downloads\yolov8n-face.pt"
model_weights_path = r"C:\Users\ishan\OneDrive\Desktop\face recog\face_match_model.pth"

# === Load data & models once ===
with open(encodings_path, "rb") as f:
    data = pickle.load(f)
face_encodings_128 = data["encodings"]
face_labels = np.array(data["names"])

embeddings_512 = np.load(embeddings_path)
labels_512 = np.load(labels_path)

mlp_model = MLP(input_dim=1024)
mlp_model.load_state_dict(torch.load(model_weights_path, map_location=device))
mlp_model.to(device)
mlp_model.eval()

yolo_model = YOLO(yolo_model_path)


# === The main face recognition function ===
def recognize_faces(image_path):
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError("Invalid image path or unable to read the image.")

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = yolo_model(image_rgb)[0]
    recognized_count = 0

    for box in results.boxes:
        conf = float(box.conf[0])
        if conf < 0.5:
            continue

        x1, y1, x2, y2 = map(int, box.xyxy[0])
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(image.shape[1], x2), min(image.shape[0], y2)
        face_location = (y1, x2, y2, x1)

        encodings = face_recognition.face_encodings(image_rgb, known_face_locations=[face_location])
        name = "Unknown"

        if encodings:
            face_encoding = encodings[0]
            distances = face_recognition.face_distance(face_encodings_128, face_encoding)
            top5_indices = distances.argsort()[:5]

            face_crop = image_rgb[y1:y2, x1:x2]
            face_crop = cv2.resize(face_crop, (160, 160))
            query_embedding = embedder.embeddings([face_crop])[0]

            scores = []
            for idx in top5_indices:
                label = face_labels[idx]
                candidate_embedding = embeddings_512[idx]
                combined = np.concatenate([query_embedding, candidate_embedding])
                input_tensor = torch.tensor(combined, dtype=torch.float32).unsqueeze(0).to(device)

                with torch.no_grad():
                    score = mlp_model(input_tensor).item()

                scores.append((score, label))
                break

            scores.sort(reverse=True, key=lambda x: x[0])
            best_score, best_label = scores[0]
            name = best_label

            actor_id = best_label.split('_')[0]

            conn = connect_to_database()
            if conn:
                cursor = conn.cursor(cursor_factory=RealDictCursor)
                celebrity_info = get_celebrity_data(actor_id)

                print(f"\n[INFO] Match Found:")
                print(f"Actor Name: {celebrity_info['name']}")
                print(f"IMDB_ID: {celebrity_info['imdb_id']}")
                print(f"Known for: {celebrity_info['known_for']}")
                print(f"About Actor: {celebrity_info['description']}\n")
                conn.close()

            recognized_count += 1

        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 1)
        cv2.putText(image, name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    print(f"Recognized {recognized_count} face(s) in the image.")

    cv2.imshow("Face Recognition Result", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


INFO:keras_facenet.embedding_model:Loading weights.
INFO:keras_facenet.utils:Looking for C:\Users\ishan/.keras-facenet\20180402-114759\20180402-114759-weights.h5


In [26]:
recognize_faces(r"C:\Users\ishan\Downloads\OIP (3).jpeg")


0: 416x640 1 face, 63.8ms
Speed: 5.2ms preprocess, 63.8ms inference, 2.6ms postprocess per image at shape (1, 3, 416, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Successfully connected to the database!
Successfully connected to the database!
Found celebrity: Johnny Depp

[INFO] Match Found:
Actor Name: Johnny Depp
IMDB_ID: nm0000136
Known for: Pirates of the Caribbean: The Curse of the Black Pearl (2003)
About Actor: John Christopher "Johnny" Depp II was born on June 9, 1963 in Owensboro, Kentucky, to Betty Sue Palmer (nÃ©e Wells), a waitress, and John Christopher Depp, a civil engineer. He was raised in Florida. He dropped out of school when he was 15, and fronted a series of music-garage bands, including one named 'The Kids'. When he married Lori A. Depp, he took a job as a ballpoint-pen salesman to support himself and his wife. A visit to Los Angeles, California, with his wife, however, happened to be a blessing in disguise, when he met up with actor , who advised him to turn to actin

INFO:keras_facenet.embedding_model:Loading weights.
INFO:keras_facenet.utils:Looking for C:\Users\ishan/.keras-facenet\20180402-114759\20180402-114759-weights.h5
Processing nm0000004_John_Belushi:   0%|          | 0/1 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


Processing nm0000007_Humphrey_Bogart:   0%|          | 0/1 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step


Processing nm0000008_Marlon_Brando:   0%|          | 0/1 [00:00<?, ?it/s]

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step


Processing nm9657438_David_Jonsson:   0%|          | 0/1 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step


Processing nm9657438_David_Jonsson: 100%|██████████| 1/1 [00:00<00:00,  6.50it/s]

Saved 916 embeddings to 'embeddingss1_json.npy' and labels to 'labelss1_json.npy'
